In [1]:
import os
if not os.path.exists("covid"):
  !git clone https://github.com/earwole1/covid covid

In [4]:
from covid.covid import Covid
c = Covid()

In [5]:
_ = c.plot_state("Florida", "cases")

In [6]:
_ = c.plot_state("New York", "deaths")